In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('/home/ayarkhan01/Desktop/Projects/stock_data_final.csv')

In [3]:
symbol = 'AAPL'
data = data[data['Symbol'] == symbol]

In [4]:
data = data.drop(['Symbol'], axis=1)
data = data.dropna()

In [5]:
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

/tmp/ipykernel_1366/889507750.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data['Date'] = pd.to_datetime(data['Date'])


In [6]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

In [7]:
sequence_length = 60  # Number of historical days to consider

X = []
y = []
for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i - sequence_length:i])
    y.append(scaled_data[i, 3])  # Using the scaled Close price as the target variable

X = np.array(X)
y = np.array(y)

In [8]:
split_ratio = 0.8  # 80% for training, 20% for testing
split_index = int(split_ratio * len(X))

X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [9]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

2024-03-06 09:01:41.267423: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-06 09:01:41.425933: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 09:01:41.426120: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 09:01:41.448652: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-06 09:01:41.526527: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-06 09:01:41.528560: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [19]:
import pandas as pd

# Define the range of epochs, batch sizes, and units to experiment with
epochs_list = [50, 100, 150]  # Example values, adjust as needed
batch_size_list = [32, 64, 128]  # Example values, adjust as needed
units_list = [32, 64, 128]  # Example values, adjust as needed

# Create an empty list to store the evaluation results
results = []

# Create a loop to iterate over different epochs, batch sizes, and units
for epochs in epochs_list:
    for batch_size in batch_size_list:
        for units in units_list:
            # Create a new instance of the model for each combination of epochs, batch size, and units
            model = Sequential()
            model.add(LSTM(units, input_shape=(sequence_length, X_train.shape[2])))
            model.add(Dense(1))

            model.compile(optimizer='adam', loss='mean_squared_error')

            # Train the model with the current epochs and batch size
            model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

            # Evaluate the model's performance on the testing set
            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mse)

            # Append the evaluation results to the list
            results.append({'Epochs': epochs, 'Batch Size': batch_size, 'Units': units, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

# Create a DataFrame from the results list
results_df = pd.DataFrame(results)

# Display the evaluation results table
print(results_df)

21/21 [==============================] - 1s 29ms/step
    Epochs  Batch Size  Units       MSE       MAE      RMSE
0       50          32     32  0.000314  0.013605  0.017714
1       50          32     64  0.000301  0.013131  0.017345
2       50          32    128  0.000580  0.019861  0.024073
3       50          64     32  0.000831  0.022401  0.028824
4       50          64     64  0.000375  0.015002  0.019355
5       50          64    128  0.000919  0.026088  0.030313
6       50         128     32  0.001286  0.029173  0.035859
7       50         128     64  0.000427  0.015544  0.020671
8       50         128    128  0.000480  0.016841  0.021913
9      100          32     32  0.000315  0.013729  0.017756
10     100          32     64  0.000305  0.013396  0.017470
11     100          32    128  0.000338  0.014288  0.018387
12     100          64     32  0.000891  0.025410  0.029845
13     100          64     64  0.000290  0.013066  0.017020
14     100          64    128  0.000627  0.020

In [21]:
average= (results_df["MSE"]+results_df['RMSE']+results_df['MAE'])/3

In [22]:
average

0     0.010544
1     0.010259
2     0.014838
3     0.017352
4     0.011577
5     0.019107
6     0.022106
7     0.012214
8     0.013078
9     0.010600
10    0.010390
11    0.011004
12    0.018715
13    0.010125
14    0.015302
15    0.013615
16    0.010966
17    0.010776
18    0.012581
19    0.012310
20    0.010956
21    0.013169
22    0.012602
23    0.024980
24    0.010744
25    0.013953
26    0.010956
dtype: float64